In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

conf = SparkConf() \
    .setAppName("batch-preprocessing-citus") \
    .set("spark.jars", "postgresql-42.7.1.jar") \
    .setMaster("local")

spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()


In [6]:
df = spark.read.csv("../datasets/batch_sample.csv", header=True, inferSchema=True)
df = df.select([col(c).alias(c.replace(" ", "_")) for c in df.columns])
reduced_df = df.select("Event_name", "Athlete_ID")
reduced_df.show()


+--------------------+----------+
|          Event_name|Athlete_ID|
+--------------------+----------+
| Selva Costera (CHI)|         0|
| Selva Costera (CHI)|         1|
|Yankee Springs 50...|        55|
|Yankee Springs 50...|        56|
|Centenary Lakes 6...|   1065546|
|Centenary Lakes 6...|   1046325|
|Sri Chinmoy Ultra...|   1615990|
|Sri Chinmoy Ultra...|   1047570|
|Millicent Six Day...|   1046124|
|Millicent Six Day...|   1046122|
+--------------------+----------+


In [7]:
jdbc_url = "jdbc:postgresql://localhost:5000/postgres"
properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}
reduced_df.write.jdbc(jdbc_url, "public.runners_reduced", mode="overwrite", properties=properties)